In [11]:
# IMPORT LIBS
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import pandas as pd
import json
import datetime
import mplfinance as mpf
import yfinance as yf
import plotly.graph_objects as go

In [12]:
#Binance OLH

def get_binance_ohlc():
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": "BTCUSDT",  # Par BTC/USDT
        "interval": "1d",     # Intervalo diario
        "limit": 10           # Últimos 10 días
    }
    response = requests.get(url, params=params)
    data = response.json()

    # Convertir a DataFrame
    df = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume", "_", "_", "_", "_", "_", "_"])
    df = df[["timestamp", "open", "high", "low", "close"]]
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

ohlc_df = get_binance_ohlc()


In [13]:
# yahoo finance OLH

# Obtener datos de los últimos 10 días para Bitcoin
data = yf.download("BTC-USD", period="6mo", interval="1h")

data.info()

[*********************100%***********************]  1 of 1 completed

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4416 entries, 2024-07-09 22:00:00+00:00 to 2025-01-09 22:00:00+00:00
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (Close, BTC-USD)   4416 non-null   float64
 1   (High, BTC-USD)    4416 non-null   float64
 2   (Low, BTC-USD)     4416 non-null   float64
 3   (Open, BTC-USD)    4416 non-null   float64
 4   (Volume, BTC-USD)  4416 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 207.0 KB


In [14]:
organized_df = data.copy()

# Organizar columnas extraidas
organized_df.columns = organized_df.columns.get_level_values(0)
selected_columns = ['Close', 'High', 'Low', 'Open', 'Volume']
organized_df = organized_df[selected_columns]
organized_df.reset_index(inplace=True)
organized_df.head()

Price,Datetime,Close,High,Low,Open,Volume
0,2024-07-09 22:00:00+00:00,57946.222656,58179.289062,57746.519531,57799.371094,0
1,2024-07-09 23:00:00+00:00,57987.714844,58051.210938,57877.222656,57940.011719,0
2,2024-07-10 00:00:00+00:00,57463.152344,58033.882812,57463.152344,58033.882812,0
3,2024-07-10 01:00:00+00:00,57776.156250,57951.242188,57423.113281,57494.210938,0
4,2024-07-10 02:00:00+00:00,57883.593750,57883.593750,57679.000000,57784.468750,0


In [15]:
organized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4416 entries, 0 to 4415
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   Datetime  4416 non-null   datetime64[ns, UTC]
 1   Close     4416 non-null   float64            
 2   High      4416 non-null   float64            
 3   Low       4416 non-null   float64            
 4   Open      4416 non-null   float64            
 5   Volume    4416 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1)
memory usage: 207.1 KB


In [16]:
#Calcular MA n
def calculate_MA(serie,n):
    return serie.rolling(window = n).mean()

organized_df['MA_10'] = calculate_MA(organized_df['Close'],10)
organized_df['MA_20'] = calculate_MA(organized_df['Close'],20)


In [20]:
df_plot = organized_df[-100:]

# Crear el gráfico de velas
fig = go.Figure(data=[go.Candlestick(
    x=df_plot['Datetime'],  # Columna de fechas
    open=df_plot['Open'],   # Columna de precios de apertura
    high=df_plot['High'],   # Columna de precios más altos
    low=df_plot['Low'],     # Columna de precios más bajos
    close=df_plot['Close']  # Columna de precios de cierre
)])

# MA 10
fig.add_trace(go.Scatter(
    x=df_plot['Datetime'],
    y=df_plot['MA_10'],
    mode='lines',
    name='Media Móvil (10 períodos)'
))

# MA 20
fig.add_trace(go.Scatter(
    x=df_plot['Datetime'],
    y=df_plot['MA_20'],
    mode='lines',
    name='Media Móvil (20 períodos)'
))

# Ajustar diseño
fig.update_layout(
    title="Gráfico de Velas de Bitcoin",
    xaxis_title="Fecha",
    yaxis_title="Precio (USD)",
    xaxis_rangeslider_visible=False
)

fig.show()


In [10]:
df_plot.describe()

Price,Close,High,Low,Open,Volume,MA_10,MA_20
count,100.000000,100.000000,100.000000,100.000000,1.000000e+02,100.000000,100.000000
mean,97319.926016,97671.389609,97014.106875,97374.834531,7.187054e+08,97561.631016,98011.921865
std,3181.028837,3104.491801,3237.746488,3141.307595,9.954547e+08,2910.095877,2282.426214
min,91929.093750,92213.171875,91232.859375,91900.656250,0.000000e+00,92728.822656,93661.213021
25%,94604.943359,94980.451172,94219.693359,94742.302734,0.000000e+00,94668.333594,95827.524219
50%,96928.160156,97186.808594,96659.562500,97040.417969,4.429076e+08,97858.648047,98334.009115
75%,99689.425781,100182.273438,99403.109375,99667.740234,9.785631e+08,99941.662305,99929.485547
max,102228.507812,102674.328125,102057.421875,102258.687500,5.847634e+09,102003.125000,101206.545833
